#Advance Python Project Assignment:

1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud
platform .


In [1]:
from flask import Flask, render_template, request
from bs4 import BeautifulSoup
import requests

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        url = request.form['url']
        if 'youtube.com' in url:
            # Scrape YouTube data
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.find('title').text
            data = {'title': title}  # Add other data as needed
        elif 'amazon.com' in url:
            # Scrape Amazon data
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.find('span', id='productTitle').text.strip()
            price = soup.find('span', class_='a-offscreen').text.strip()
            data = {'title': title, 'price': price}  # Add other data as needed
        else:
            data = {'error': 'Unsupported website.'}
        return render_template('result.html', data=data)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)  # Set debug=False for production



'''

html file(index.html) - 1



<!DOCTYPE html>
<html>
<head>
    <title>Web Scraper</title>
</head>
<body>
    <h1>Enter a URL to scrape:</h1>
    <form method="POST">
        <input type="text" name="url" placeholder="Enter URL">
        <button type="submit">Scrape</button>
    </form>
</body>
</html>


html file(result.html) - 2



<!DOCTYPE html>
<html>
<head>
    <title>Scraped Data</title>
</head>
<body>
    <h1>Scraped Data:</h1>
    {% if data.error %}
        <p>{{ data.error }}</p>
    {% else %}
        {% for key, value in data.items() %}
            <p><b>{{ key }}:</b> {{ value }}</p>
        {% endfor %}
    {% endif %}
</body>
</html>

'''

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


2. Create a Flask app that consumes data from external APIs and displays it to users.
Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform.

In [ ]:
from flask import Flask, render_template, request
import requests

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        city = request.form['city']
        api_key = 'YOUR_API_KEY'  # Replace with your OpenWeatherMap API key
        url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
        response = requests.get(url)
        data = response.json()
        return render_template('weather.html', data=data)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


'''
html file(index.html) - 1


<!DOCTYPE html>
<html>
<head>
    <title>Weather App</title>
</head>
<body>
    <h1>Enter a city:</h1>
    <form method="POST">
        <input type="text" name="city" placeholder="Enter city">
        <button type="submit">Get Weather</button>
    </form>
</body>
</html>


html file(weather.html) - 2


<!DOCTYPE html>
<html>
<head>
    <title>Weather Result</title>
</head>
<body>
    <h1>Weather in {{ data.name }}:</h1>
    <p>Temperature: {{ data.main.temp }}°C</p>
    <p>Description: {{ data.weather[0].description }}</p>
</body>
</html>


'''


3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.

In [ ]:
from flask import Flask, redirect, url_for, render_template
from flask_dance.contrib.google import make_google_blueprint, google
from flask_dance.contrib.facebook import make_facebook_blueprint, facebook
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config["SECRET_KEY"] = "your-secret-key" # Replace with your secret key
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///oauth.db" # Database for storing OAuth tokens
db = SQLAlchemy(app)

# OAuth blueprints
google_bp = make_google_blueprint(
    client_id="your-google-client-id", # Replace with your Google client ID
    client_secret="your-google-client-secret", # Replace with your Google client secret
    scope=["profile", "email"],
    redirect_url="/google-auth",
)
app.register_blueprint(google_bp, url_prefix="/login")

facebook_bp = make_facebook_blueprint(
    client_id="your-facebook-app-id", # Replace with your Facebook App ID
    client_secret="your-facebook-app-secret", # Replace with your Facebook App Secret
    scope=["public_profile", "email"],
    redirect_url="/facebook-auth",
)
app.register_blueprint(facebook_bp, url_prefix="/login")

# OAuth token storage model
class OAuth(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    provider = db.Column(db.String(50), nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    token = db.Column(db.JSON, nullable=False)

with app.app_context():
    db.create_all()


@app.route("/google-auth")
def google_auth():
    if not google.authorized:
        return redirect(url_for("google.login"))
    account_info = google.get("/oauth2/v2/userinfo")
    if account_info.ok:
        # Store OAuth token in the database
        oauth = OAuth(provider="google", token=account_info.json())
        db.session.add(oauth)
        db.session.commit()
        return redirect(url_for("index"))
    return "Failed to fetch user info."

@app.route("/facebook-auth")
def facebook_auth():
    if not facebook.authorized:
        return redirect(url_for("facebook.login"))
    account_info = facebook.get("/me?fields=id,name,email")
    if account_info.ok:
        # Store OAuth token in the database
        oauth = OAuth(provider="facebook", token=account_info.json())
        db.session.add(oauth)
        db.session.commit()
        return redirect(url_for("index"))
    return "Failed to fetch user info."


<a href="{{ url_for('google.login') }}">Login with Google</a>
<a href="{{ url_for('facebook.login') }}">Login with Facebook</a>





4. Develop a recommendation system using Flask that suggests content to users based on their preferences.

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Sample user-item matrix
data = {
    'user_id': [1, 2, 3, 4, 5],
    'item_id': [[1, 2, 3], [2, 3, 4], [1, 3, 5], [2, 4, 5], [1, 4, 5]],
    'rating': [[5, 4, 3], [4, 5, 3], [5, 3, 4], [4, 3, 5], [5, 4, 3]]
}
ratings_data = pd.DataFrame(data)

# Function to calculate recommendations
def get_recommendations(user_id):
    # Create a user-item matrix (replace with your actual matrix)
    user_item_matrix = pd.DataFrame(0, index=ratings_data['user_id'].unique(), columns=range(1, 6))
    for index, row in ratings_data.iterrows():
        for i, item_id in enumerate(row['item_id']):
            user_item_matrix.loc[row['user_id'], item_id] = row['rating'][i]

    # Calculate cosine similarity
    item_similarity = cosine_similarity(user_item_matrix.T)

    # Get user ratings
    user_ratings = user_item_matrix.loc[user_id]

    # Get similar items
    similar_items = item_similarity[user_ratings[user_ratings > 0].index]

    # Calculate item scores
    item_scores = similar_items.sum(axis=0)
    item_scores = pd.Series(item_scores, index=user_item_matrix.columns)
    item_scores = item_scores.sort_values(ascending=False)

    # Recommend items not yet rated by the user
    recommended_items = item_scores[~user_ratings.isin(user_ratings[user_ratings > 0])].head(3)

    return recommended_items.index.tolist()

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        user_id = int(request.form['user_id'])
        recommendations = get_recommendations(user_id)
        return render_template('recommendations.html', recommendations=recommendations, user_id=user_id)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


'''
html file(index.html) - 1


<!DOCTYPE html>
<html>
<head>
    <title>Recommendation System</title>
</head>
<body>
    <h1>Enter User ID:</h1>
    <form method="POST">
        <input type="number" name="user_id" placeholder="Enter User ID">
        <button type="submit">Get Recommendations</button>
    </form>
</body>
</html>


html file(recommendations.html) - 2

<!DOCTYPE html>
<html>
<head>
    <title>Recommendation System</title>
</head>
<body>
    <h1>Enter User ID:</h1>
    <form method="POST">
        <input type="number" name="user_id" placeholder="Enter User ID">
        <button type="submit">Get Recommendations</button>
    </form>
</body>
</html>

'''




